In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import catboost
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
df=pd.read_csv('../input/churn-risk-rate-hackerearth-ml/train.csv',na_values=['?','-999','Error','xxxxxxxx'])
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
import pandas_profiling

In [ ]:
# df.profile_report()

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
df=df.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)

In [ ]:
df['churn_risk_score'].value_counts()

In [ ]:
def fillNan(df, col, value):
    df[col].fillna(value, inplace=True)
    
fillNan(df, 'region_category', df['region_category'].mode()[0])
fillNan(df, 'points_in_wallet', df['points_in_wallet'].mean())
fillNan(df, 'preferred_offer_types', df['preferred_offer_types'].mode()[0])
# fillNan(df, 'joined_through_referral', 'No')
# fillNan(df, 'medium_of_operation', 'Desktop')

In [ ]:
df.isnull().sum()

In [ ]:
for i in df.columns:
  if df[i].dtype=='float64':
    df[i]=df[i].fillna(df[i].mean())
  else:
    df[i]=df[i].fillna(method='ffill')

In [ ]:
df.isnull().sum().sum()

In [ ]:
# df['joining_date']=df['joining_date'].apply(lambda x:int(x.split('-')[0]))
# df.rename(columns={'joining_date':'joining_year'},inplace=True)

df['joining_date'] = pd.to_datetime(df['joining_date'])
df['days_since_joined'] = df['joining_date'].apply(lambda x:(pd.Timestamp('today') - x).days)

df.drop(['joining_date'], inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
df1=pd.get_dummies(df,drop_first=True)

In [ ]:
x=df1.drop('churn_risk_score',axis=1)
y=df1['churn_risk_score']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,stratify=y,random_state=42)

In [ ]:
rd=RandomForestClassifier()

In [ ]:
rd.fit(x_train,y_train)

In [ ]:
rdpred=rd.predict(x_test)

In [ ]:
print(classification_report(y_test,rdpred))

In [ ]:
print(f1_score(y_test,rdpred,average='macro'))

In [ ]:
import xgboost

In [ ]:
xgc=xgboost.XGBClassifier()

In [ ]:
xgc.fit(x_train,y_train)

In [ ]:
xgcpred=xgc.predict(x_test)

In [ ]:
print(f1_score(y_test,xgcpred,average='macro'))

In [ ]:
# params={'learning_rate':[0.05,0.1,0.15,0.2],
#       'max_depth':[3,4,5,6,8,10],
#       'min_child_weight':[1,3,5,7],
#       'gamma':[0.0,0.1,0.2,0.3,0.4,0.5],
#       'colsample_bytree':[0.3,0.4,0.5,0.7]}
# rsearch=RandomizedSearchCV(xgc,param_distributions=params,n_iter=100,cv=3,random_state=42,n_jobs=-1)
# rsearch.fit(x,y)
# rsearch.best_parameter_ 

In [ ]:
# best_estimaters=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=0.7, gamma=0.4,
#              learning_rate=0.1, max_delta_step=0, max_depth=4,
#             min_child_weight=3, missing=None, n_estimators=119, n_jobs=1,
#             nthread=None, objective='multi:softprob', random_state=0,
#             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#             silent=None, subsample=1, verbosity=1)

In [ ]:
xgc=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.4,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=3, missing=None, n_estimators=119, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgc.fit(x_train,y_train)

In [ ]:
xgcpred=xgc.predict(x_test)

In [ ]:
print(f1_score(y_test,xgcpred,average='macro'))

In [ ]:
xgc.fit(x,y)

In [ ]:
test=pd.read_csv('../input/churn-risk-rate-hackerearth-ml/test.csv',na_values=['?','-999','Error','xxxxxxxx'])
test.head()

In [ ]:
test=test.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)

In [ ]:
def fillNan(df, col, value):
    df[col].fillna(value, inplace=True)
    
fillNan(test, 'region_category', test['region_category'].mode()[0])
fillNan(test, 'points_in_wallet', test['points_in_wallet'].mean())
fillNan(test, 'preferred_offer_types', test['preferred_offer_types'].mode()[0])
# fillNan(test, 'joined_through_referral', 'No')
# fillNan(test, 'medium_of_operation', 'Desktop')

In [ ]:
for i in test.columns:
  if test[i].dtype=='float64':
    test[i]=test[i].fillna(test[i].mean())
  else:
    test[i]=test[i].fillna(method='ffill')

In [ ]:
# test['joining_date']=test['joining_date'].apply(lambda x:int(x.split('-')[0]))
# test.rename(columns={'joining_date':'joining_year'},inplace=True)

test['joining_date'] = pd.to_datetime(test['joining_date'])
test['days_since_joined'] = test['joining_date'].apply(lambda x:(pd.Timestamp('today') - x).days)

test.drop(['joining_date'], inplace=True, axis=1)

In [ ]:
test1=pd.get_dummies(test,drop_first=True)

In [ ]:
rdpred=xgc.predict(test1)

In [ ]:
df2=pd.read_csv('../input/churn-risk-rate-hackerearth-ml/test.csv',usecols=['customer_id'])

In [ ]:
df2['churn_risk_score']=rdpred

In [ ]:
df2.to_csv('xgb.csv', index=False)
